## AAM / Machine Learning - Atividade 02 - Competição Porto Seguro

**Equipe**:

*Ciro Mora* - RA: 111310

*José Diniz* - RA: 183134

*Renan Renger* - RA: 183148

*Roberto Rodrigues* -  RA: 060235

**Projeto**

Predição de acionamento ou não de sinistro de seguro baseado em dataset liberado pela competição.

**Observação**

Boa parte dos comentário do notebook são proveniente de um notebook base fornecido em sala com a descrição da atividade serão mantidos por comodidade.


## Pré-processamento é provavelmente a parte mais importante de ciência dos dados

Ter dados representativos sem atributos faltantes é provavelmente o pote de ouro em ciência dos dados. É muito incomum que os dados do mundo real não apresentem anomalias seja da própria natureza ou sejam anomalias introduzidas no processo de medição e registro da observação (amostra).

Esse notebook é voltado para como tratar dados mais complexos e transformar todas as informações em números que façam sentido para que o modelo seja capaz de traçar a relação entre atributos e classes. A seguir é oferecida uma pequena parcela de um conjunto de dados da empresa Porto Seguro, no qual uma competição foi aberta e os competidores foram desafiados a criar um modelo para prever se uma apólice teria um sinistro registrado ou não, indicando o uso do serviço.

Algumas características sobre o nome das features:
- O nome dos atributos indica o grupo ao qual pertence (ind, reg, car);
- Os prefixos bin e cat indicam atributos binários e categóricos, respectivamente;
- Atributos sem os prefixos citados podem ser ordinais ou contínuos;
- Atributos com -1 indicam dado faltante (missing); e
- A coluna 'target' indica se houve sinistro para apólice ou não.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import warnings
warnings.filterwarnings('ignore')

from numba import jit

In [ ]:
@jit
def eval_gini(y_true, y_prob):
    """
    Original author CPMP : https://www.kaggle.com/cpmpml
    In kernel : https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [ ]:
dfTreino = pd.read_csv('../input/train.csv')
dfTeste = pd.read_csv('../input/test.csv')

In [ ]:
# Podemos usar a coluna "id" como o índice dos Datasets, sem nenhum prejuizo
dfTreino.set_index('id', inplace=True)
dfTeste.set_index('id', inplace=True)

In [ ]:
headNumber = 5
print(f'Dataset de treino - Primeiras {headNumber} linhas')
display(dfTreino.head(headNumber))

print(f'Dataset de teste - Primeiras {headNumber} linhas')
display(dfTeste.head(headNumber))

In [ ]:
print('Dataset de treino - Estatistica descritiva')
display(dfTreino.describe())

print('Dataset de teste - Estatistica descritiva')
display(dfTeste.describe())

In [ ]:
print('Dataset de treino - Sumário das Features')
print(dfTreino.info())
print('---')
print('Dataset de treino - Sumário das Features')
print(dfTeste.info())

In [ ]:
print(f'Dataset de treino tem {dfTreino.shape[0]} linhas por {dfTreino.shape[1]} colunas ({dfTreino.shape[0] * dfTreino.shape[1]} celulas)')
print(f'Dataset de teste tem {dfTeste.shape[0]} linhas por {dfTeste.shape[1]} colunas ({dfTeste.shape[0] * dfTeste.shape[1]} celulas)')

In [ ]:
nonUsed, used = dfTreino.groupby('target').size()
print(f'Das {dfTreino.shape[0]} entradas no dataset, {nonUsed} foram de casos onde não foi acionado o seguro e {used} foram caso onde houve acionamento')
print(f'Temos assim {round((used/nonUsed) * 100,6)}% de ocorrencias em que o resultado (1 ou "houve acionamento") desejamos prever')

## Observação
Com base nas analises acima, podemos perceber algumas coisas:

1 -  Os valores estão desnormalizados, variando tanto de tipo (numéricos discretos e continuos, categoricos e binários);

2 - Faltando dados em ambos os datasets (marcados como -1 nos datasets);

3 - **Temos um enorme desbalanço no que tange a ocorrencias cujo valor é desejado ("houve acionamento") vs ocorrencias sem acionamento**

---

In [ ]:
print(f'Antes - Treino tem {dfTreino.shape[0]} linhas por {dfTreino.shape[1]} colunas ({dfTreino.shape[0] * dfTreino.shape[1]} celulas)')
dfTreino.drop_duplicates()
print(f'Depois - Treino tem {dfTreino.shape[0]} linhas por {dfTreino.shape[1]} colunas ({dfTreino.shape[0] * dfTreino.shape[1]} celulas)')
print('---')
print(f'Antes - Teste tem {dfTeste.shape[0]} linhas por {dfTeste.shape[1]} colunas ({dfTeste.shape[0] * dfTeste.shape[1]} celulas)')
dfTeste.drop_duplicates()
print(f'Depois - Teste tem {dfTeste.shape[0]} linhas por {dfTeste.shape[1]} colunas ({dfTeste.shape[0] * dfTeste.shape[1]} celulas)')

Ao trabalhar com as colunas (atributos), é interessante ter uma organização de que tipo de dado determinado atributo é, e para quais propósitos determinado atributo pode ser usado. Nesse sentido, seguindo o trabalho de https://www.kaggle.com/bertcarremans/data-preparation-exploration, vamos criar metadados para esse conjunto.

In [ ]:
def generateMetadata(dfInput):
    data = []
    for f in dfInput.columns:
        # definindo o uso (entre rótulo, id e atributos)
        if f == 'target':
            role = 'target' # rótulo
        elif f == 'id':
            role = 'id'
        else:
            role = 'input' # atributos

        # definindo o tipo do dado
        if 'bin' in f or f == 'target':
            level = 'binary'
        elif 'cat' in f or f == 'id':
            level = 'nominal'
        elif dfInput[f].dtype == float or dfInput[f].dtype == np.float64:
            level = 'interval'
        elif dfInput[f].dtype == int or dfInput[f].dtype == np.int64:
            level = 'ordinal'
            
        # mantem keep como verdadeiro pra tudo, exceto id
        keep = True
        if f == 'id':
            keep = False

        # cria o tipo de dado
        dtype = dfInput[f].dtype

        # cria dicionário de metadados
        f_dict = {
            'varname': f,
            'role': role,
            'level': level,
            'keep': keep,
            'dtype': dtype
        }
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
    meta.set_index('varname', inplace=True)
    
    return meta

Para visualizar o atributo e todos seus metadados, basta mostrar a variável meta:

In [ ]:
meta_train = generateMetadata(dfTreino)
meta_test = generateMetadata(dfTeste)

In [ ]:
display(meta_train)
display(meta_test)

Com essa estrutura de metadados, fica fácil consultar quais colunas quer se manter e que são nominais, por exemplo:

In [ ]:
print('Metadados categoricos da base de treino')
print(meta_train[(meta_train.level == 'nominal') & (meta_train.keep)].index)
print('---')
print('Metadados categoricos da base de teste')
print(meta_test[(meta_test.level == 'nominal') & (meta_test.keep)].index)

Da mesma forma, seria possível contar os atributos por tipo de uso e dado:

In [ ]:
print('Tipos e quantidade de features do dataset de treino')
display(pd.DataFrame({'count' : meta_train.groupby(['role', 'level'])['role'].size()}).reset_index())

print('Tipos e quantidade de features do dataset de teste')
display(pd.DataFrame({'count' : meta_test.groupby(['role', 'level'])['role'].size()}).reset_index())

## Valores faltantes

Conforme já mencionado, os valores faltantes são indicados por -1, então é importante saber quais colunas têm valores faltantes e em qual proporção.

In [ ]:
def getMissingAttributes(dfInput):
    atributos_missing = []
    return_missing = []

    for f in dfInput.columns:
        missings = dfInput[dfInput[f] == -1][f].count()
        if missings > 0:
            atributos_missing.append(f)
            missings_perc = missings/dfInput.shape[0]
            
            return_missing.append([f, missings, missings_perc])

            print('Atributo {} tem {} amostras ({:.2%}) com valores faltantes'.format(f, missings, missings_perc))
            

    print('No total, há {} atributos com valores faltantes'.format(len(atributos_missing)))
    
    return pd.DataFrame(return_missing).rename(index=str, columns={0: "column_name", 1: "column_nulls", 2: "column_percentage"})

In [ ]:
missing_Train = getMissingAttributes(dfTreino)
display(missing_Train)

In [ ]:
missing_Test = getMissingAttributes(dfTeste)
display(missing_Test)

Duas estratégias podem ser optadas aqui: simplesmente remover o atributo ou tentar preenchê-lo de forma sintética. Preencher de forma sintética pode gerar uma falsa distribuição quando o número de atributos faltantes é muito alto. Quando este for o caso, é sempre seguro optar por remover o atributo inteiro. Também é importante lembrar que a estratégia de preenchimento deve ser coerente com o tipo de dado, por exemplo: **dados ordinais não devem ser preenchidos com média, nem dados contínuos com moda.**

In [ ]:
# limiar de remoção - 42.5% de nulos
remove_threshold = 0.425

In [ ]:
columns_to_remove = np.array(missing_Train.column_name[(missing_Train.column_percentage >= remove_threshold)])

In [ ]:
# removendo as colunas que tem muitos valores faltantes
dfTreino = dfTreino.drop(columns_to_remove, axis=1)
dfTeste = dfTeste.drop(columns_to_remove, axis=1)

# atualiza os metadados para ter como referência
meta_train.loc[(columns_to_remove),'keep'] = False  
meta_test.loc[(columns_to_remove),'keep'] = False

# remove do frame de colunas com falta de dados as colunas que foram dropadas
missing_Train.drop(missing_Train[(np.isin(missing_Train.column_name, columns_to_remove))].index)

In [ ]:
# Usa ou moda ou média para preencher os valores "vazios" que nosso dataset contem, baseado nos metadados do mesmo
def fillNullNumbers(dfInput, dfMetadata, dfMissing, missing_default, label):

    from sklearn.impute import SimpleImputer

    media_imp = SimpleImputer(missing_values=missing_default, strategy='mean')
    moda_imp = SimpleImputer(missing_values=missing_default, strategy='most_frequent')

    for index,row in dfMissing.iterrows():
        columnName = row['column_name']
        columnType = dfMetadata.level[(dfMetadata.index == columnName)][0]

        if (columnType == 'interval'):
            imputerToUse = media_imp
            imputerString = 'media_imp'
        elif (columnType == 'ordinal'):
            imputerToUse = moda_imp
            imputerString = 'moda_imp'
        else:
            imputerToUse = None
            imputerString = None

        if (imputerToUse != None):
            dfInput[columnName] = imputerToUse.fit_transform(dfInput[[columnName]]).ravel()
            print(f"{label} - Preenchida coluna {columnName}, cujo tipo é {columnType}, usando o Imputer {imputerString}")

    return dfInput

In [ ]:
dfTreino = fillNullNumbers(dfTreino, meta_train, missing_Train, -1, 'Treino')
print('---')
dfTeste = fillNullNumbers(dfTeste, meta_test, missing_Train, -1, 'Teste')

Os atributos categóricos podem ser mantidos porque o número de valores faltantes não é expressivo. Inclusive, a estratégia de preenchimento dos **atributos categóricos** é sempre mais complexa. Esses atributos **não se beneficiam de medidas estatísticas** como moda e média, portanto essas medidas não servem para preenchê-los de forma sintética.

---

## One-hot encoding (ou dummy variables)

Depois de ter tratado os dados faltantes, é importante que os dados ordinais tenham representação apropriada para o problema tratado. Se o dado não tem distância ou rankamento entre eles, cada valor de um atributo deve ser representado por um conjunto de atributos de mesma distância. *(Verificar slides desse encontro para que isso fique mais claro)*

Os dados que precisam ser separados em mais dimensões já foram identificados como nominais no pré-processamento. É importante verificar se esses dados têm grande variedade de valores ou não, e aplicar essa separação apenas se for viável. Por exemplo, se um determinado atributo tem 300 valores, isso geraria 300 colunas novas. Isso só se justificaria se fosse uma base realmente grande e se houvesse uma correlação muito alta entre essa variedade de valores e a classe.

In [ ]:
def performOneHotEncoding(dfTrain, dfTest, meta_generic, dist_limit):
    v = meta_generic[(meta_generic.level == 'nominal') & (meta_generic.keep)].index
    display(v)
    for f in v:
        dist_values = dfTrain[f].value_counts().shape[0]
        print('Atributo {} tem {} valores distintos'.format(f, dist_values))
        if (dist_values > dist_limit):
            print('Atributo {} tem mais de {} valores distintos e por isso será ignorado'.format(f, dist_limit))
            dfTrain.drop([f], axis=1)
            v = v.drop([f])
        
    print('Antes do one-hot encoding tinha-se {} atributos'.format(dfTrain.shape[1]))
    dfTrain = pd.get_dummies(dfTrain, columns=v, drop_first=True)
    print('Depois do one-hot encoding tem-se {} atributos'.format(dfTrain.shape[1]))

    dfTest = pd.get_dummies(dfTest, columns=v, drop_first=True)
    missing_cols = set( dfTrain.columns ) - set( dfTest.columns )
    for c in missing_cols:
        dfTest[c] = 0

    dfTrain, dfTest = dfTrain.align(dfTest, axis=1)
    
    return dfTrain, dfTest

Vamos optar por manter todos atributos e, portanto, gerar o conjunto de atributos que os mantêm à mesma distância:

In [ ]:
dfTreino, dfTeste = performOneHotEncoding(dfTreino, dfTeste, meta_train, 200)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

dfTreino[dfTreino.columns] = min_max_scaler.fit_transform(dfTreino[dfTreino.columns])
dfTeste[dfTeste.columns] = min_max_scaler.fit_transform(dfTeste[dfTeste.columns])

In [ ]:
dfTeste.drop(['target'], axis=1, inplace=True)

## Depois de todo pré-processamento...

É hora de verificar se tanto treino como teste têm o mesmo tamanho/formato, e aplicar um modelo de classificação já que esse é um problema desse tipo. Vale lembrar que o tamanho do treino e teste pode variar quando você estiver participando de outras competições ou explorando outros conjuntos de dados.

Isso porque na maioria das competições não se tem o *target* do test. Estima-se uma resposta e submete ao Kaggle, por exemplo, para que ele verifique qual foi o resultado final. Então esse tamanho pode variar em 1 entre treino e teste. No nosso caso, como todos os dados vêm de uma mesma fonte para experimentos, é esperado que tenham a mesma quantidade de atributos ou colunas.

In [ ]:
# Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Feature Selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, ShuffleSplit

# Auxiliary Scores
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [ ]:
def showDistribution(val_classes):
    nonUsed, used = pd.DataFrame(val_classes).groupby('target').size()
    print('---')
    print(f'Das {pd.DataFrame(val_classes).shape[0]} entradas no dataset, {nonUsed} foram de casos onde não foi acionado o seguro e {used} foram caso onde houve acionamento')
    print(f'Temos assim {round((used/len(val_classes)) * 100,6)}% de ocorrencias em que o resultado (1 ou "houve acionamento") desejamos prever')
    print('---')

In [ ]:
def logisticRegression(X_Train, y_Train, X_Val, y_Val):

    model = LogisticRegression(solver='lbfgs')

    model.fit(X_Train, y_Train)

    y_pred_class = model.predict(X_Val)
    y_pred_proba = model.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)
    gini = eval_gini(y_Val, y_pred_class)

    print(f'Baseline - Regressão Logistica')
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return model, 'Baseline - Regressão Logistica'

In [ ]:
# n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 70, random_state = 0 - Recall: 0.057621%

def xGBClassifier(X_Train, y_Train, X_Val, y_Val, modelName, modelParams):

    if (modelParams == None):
        clf = XGBClassifier()
    else:
        clf = XGBClassifier(**modelParams)  
        modelName = modelName + ' - Parameters: ' + str(modelParams)
    
    clf.fit(X_Train, y_Train)

    y_pred_class = clf.predict(X_Val)
    y_pred_proba = clf.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    gini = eval_gini(y_Val, y_pred_class)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)

    print(modelName)
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return clf, modelName

In [ ]:
def decisionTreeClassifier(X_Train, y_Train, X_Val, y_Val):

    clf = DecisionTreeClassifier()

    clf.fit(X_Train, y_Train)

    y_pred_class = clf.predict(X_Val)
    y_pred_proba = clf.predict_proba(X_Val)

    recall = recall_score(y_Val, y_pred_class)
    accuracy = accuracy_score(y_Val, y_pred_class)
    gini = eval_gini(y_Val, y_pred_class)
    logloss = log_loss(y_Val, y_pred_proba)
    precision =  precision_score(y_Val, y_pred_class)
    f1 = f1_score(y_Val, y_pred_class)

    print(f'Decision Tree - Default Parameters')
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
    print('---')
    
    return clf, f'Decision Tree - Default Parameters'

In [ ]:
def gridSearchKNN(X_Train, y_Train, X_Val, y_Val, k_range):
    clf=KNeighborsClassifier()
    param_grid=dict(n_neighbors=k_range)
    scores = ['f1']
    for sc in scores:
        grid=GridSearchCV(clf,param_grid,cv=4,scoring=sc,n_jobs=-1)
        print("K-Nearest Neighbors - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)
        y_pred_proba = grid.predict_proba(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        logloss = log_loss(y_Val, y_pred_proba)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'KNN with recall-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'Log Loss: {round(logloss, 6)}')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'KNN with recall-maxing hyperparameters - {grid.best_params_}'

In [ ]:
def gridSearchSVC(X_Train, y_Train, X_Val, y_Val):
    svc=SVC()
    param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5],'C': [1, 10, 100, 1000]},
                  {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    scores = ['f1']
    for sc in scores:
        grid=GridSearchCV(svc,param_grid,cv=4,scoring=sc,n_jobs=-1)
        
        print("Support Vector Classifier - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'SVC with recall-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'SVC with recall-maxing hyperparameters - {grid.best_params_}'

In [ ]:
def gridSearchXGB(X_Train, y_Train, X_Val, y_Val, score):
    xgb=XGBClassifier(random_state = 0)
    param_grid = [{'n_estimators': [100, 200, 300, 400], 'learning_rate': [0.1, 0.25, 0.5, 0.75],'max_depth': [25, 50, 75, 100], 'gamma': [0, 3, 6, 9]}]
    scores = [score]
    for sc in scores:
        grid=GridSearchCV(xgb,param_grid,cv=2,scoring=sc,n_jobs=-1)
        
        print("XGBoost - Tuning hyper-parameters for %s" % sc)
        
        grid.fit(X_Train,y_Train)
        print(grid.best_params_)
        print(np.round(grid.best_score_,3))
        
        y_pred_class = grid.predict(X_Val)

        recall = recall_score(y_Val, y_pred_class)
        accuracy = accuracy_score(y_Val, y_pred_class)
        gini = eval_gini(y_Val, y_pred_class)
        precision =  precision_score(y_Val, y_pred_class)
        f1 = f1_score(y_Val, y_pred_class)

        print(f'XGBoost with {sc}-maxing hyperparameters - {grid.best_params_}')
        print('---')
        print(f'Acurácia: {round(accuracy, 6)}%')
        print(f'Recall: {round(recall, 6)}%')
        print(f'Precisão: {round(precision, 6)}%')
        print(f'F1 Score: {round(f1, 6)}')
        print(f'Gini: {round(gini, 6)}')

        print('---')
        print('Matriz de Confusão')
        display(pd.DataFrame(confusion_matrix(y_Val, y_pred_class)))
        print('---')
        
        return grid, f'XGBoost with {sc}-maxing hyperparameters - {grid.best_params_}'

In [ ]:
def predictTestDataset(X_Test, y_Test, clfModel, clfName):
    y_pred_class = clfModel.predict(X_Test)
    y_pred_proba = clfModel.predict_proba(X_Test)

    recall = recall_score(y_Test, y_pred_class)
    accuracy = accuracy_score(y_Test, y_pred_class)
    gini = eval_gini(y_Test, y_pred_class)
    logloss = log_loss(y_Test, y_pred_proba)
    precision =  precision_score(y_Test, y_pred_class)
    f1 = f1_score(y_Test, y_pred_class)

    print(clfName)
    print('---')
    print(f'Acurácia: {round(accuracy, 6)}%')
    print(f'Recall: {round(recall, 6)}%')
    print(f'Precisão: {round(precision, 6)}%')
    print(f'Log Loss: {round(logloss, 6)}')
    print(f'F1 Score: {round(f1, 6)}')
    print(f'Gini: {round(gini, 6)}')

    print('---')
    print('Matriz de Confusão')
    display(pd.DataFrame(confusion_matrix(y_Test, y_pred_class)))
    print('---')

In [ ]:
def predictContestDataset(X_Test, clfModel, clfName):
    
    print(clfName)
    print('---')
    
    y_pred_class = clfModel.predict(X_Test)
    y_pred_proba = clfModel.predict_proba(X_Test)
    
    pd_prediction = pd.DataFrame(y_pred_class)
    pd_prediction.columns = ['target']
    showDistribution(pd_prediction)

    return y_pred_class, y_pred_proba

In [ ]:
print(dfTreino.shape)
print(dfTeste.shape)

In [ ]:
sample_size = 10000
inactive_sample_size = int(sample_size * 1)

activated_indices = dfTreino[dfTreino.target == 1].index
activated = dfTreino.loc[np.random.choice(activated_indices, sample_size, replace=False)]

inactive_indices = dfTreino[dfTreino.target == 0].index
inactive = dfTreino.loc[np.random.choice(inactive_indices, inactive_sample_size, replace=False)]

subsampled = pd.concat([activated, inactive])

subsampled.sort_index(inplace=True)

In [ ]:
X = subsampled.drop(['target'], axis=1)
y = subsampled['target']

In [ ]:
X_supersampled = dfTreino.drop(X.index).drop(['target'], axis=1)
y_supersampled = dfTreino.drop(X.index)['target']

In [ ]:
print(X_supersampled.shape)
print(y_supersampled.shape)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
showDistribution(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

showDistribution(y_train)
logRegModel, logRegName = logisticRegression(X_train, y_train, X_val, y_val)
xgbPureModel, xgbPureName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Base',None)
#xgbPresetModel, xgbPresetName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Preset', {'n_estimator':400, 'learning_rate' : 0.5,'random_state' : 0,'max_depth':70,'objective':"binary:logistic",'subsample':.8,'min_child_weig':6,'colsample_bytr':.8,'scale_pos_weight':1.6, 'gamma':10, 'reg_alph':8, 'reg_lambda':1})
xgbHyperParametrizedModel, xgbHyperParametrizedName = xGBClassifier(X_train, y_train, X_val, y_val, 'XGBoost - Hyperparametrized',{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 2, 'min_child_weight': 16, 'n_estimators': 100, 'subsample': 1.0})
#decTreeModel, decTreeName = decisionTreeClassifier(X_train, y_train, X_val, y_val)
#knnModel, knnName = gridSearchKNN(X_train, y_train, X_val, y_val, list(range(1,20)))
#svcModel, svcName = gridSearchSVC(X_train, y_train, X_val, y_val)

showDistribution(y_supersampled)
predictTestDataset(X_supersampled, y_supersampled, logRegModel, logRegName)
predictTestDataset(X_supersampled, y_supersampled, xgbPureModel, xgbPureName)
#predictTestDataset(X_supersampled, y_supersampled, xgbPresetModel, xgbPresetName)
predictTestDataset(X_supersampled, y_supersampled, xgbHyperParametrizedModel, xgbHyperParametrizedName)
#predictTestDataset(X_supersampled, y_supersampled, decTreeModel, decTreeName)
#predictTestDataset(X_supersampled, y_supersampled, knnModel, knnName)
#predictTestDataset(X_supersampled, y_supersampled, svcModel, svcName)

In [ ]:
#showDistribution(y_train)
#xgbGridSearchModel, xgbGridSearchName = gridSearchXGB(X_train, y_train, X_val, y_val, 'f1')

#showDistribution(y_supersampled)
#predictTestDataset(X_supersampled, y_supersampled, xgbGridSearchModel, xgbGridSearchName)

In [ ]:
contest_prediction, contest_prediction_probability = predictContestDataset(dfTeste, xgbHyperParametrizedModel, xgbHyperParametrizedName)

In [ ]:
sample    = pd.read_csv('../input/sample_submission.csv', low_memory=False)
sample.target = contest_prediction_probability
sample.target = 1 - sample.target
sample.to_csv("submission.csv", float_format='%.6f', index=False)